# Game Record Setup

TODO: Need to rethink all steps for proper formulas and calculations.

## Table of Contents

* [Data Setup](#data_setup)  
* [Model Setup](#model_setup)  
* [Game Records Table Creation](#game_records)
    * [Win Percentage](#win_pct)
    * [Expected Value](#ev)
    * [Game Score](#game_score)
    * [Recommended Bet Amount](#rec_bet)
* [Save to Database](#save)

## Imports and Global Settings

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
from tensorflow import keras
from sqlalchemy import create_engine
from pycaret.regression import *

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

<a id=data_setup></a>
## Database Connection

In [2]:
username = 'postgres'
password = None
endpoint = None
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

## Load Data

TODO: Ability to input date or dates to be loaded.

In [3]:
query = """
SELECT cdi.game_id,
 cdi.game_date,
 cdi.home_team,
 cdi.away_team,
 cdi.home_score,
 cdi.away_score,
 cdi.home_result,
 cdi.covers_game_url,
 cdi.league_year,
 cdi.home_spread,
 cdi.home_spread_result,
 cdi.pred_date,
 cdi.open_line_home,
 cdi.open_line_away,
 cdi.fd_line_price_home,
 cdi.fd_line_away,
 cdi.fd_line_price_away,
 cdi.dk_line_price_home,
 cdi.dk_line_away,
 cdi.dk_line_price_away,
 cdi.covers_consenses_away,
 mr.home_team_num,
 mr.away_team_num,
 mr.league_year_end,
 mr.home_line,
 mr.fd_line_home,
 mr.dk_line_home,
 mr.covers_consenses_home,
 mr.wins,
 mr.losses,
 mr.win_pct,
 mr.expected_wins,
 mr.expected_losses,
 mr.home_ppg,
 mr.home_papg,
 mr.away_wins,
 mr.away_losses,
 mr.away_win_pct,
 mr.away_expected_wins,
 mr.away_expected_losses,
 mr.away_ppg,
 mr.away_papg,
 mr.g,
 mr.mp,
 mr.pts,
 mr.ast,
 mr.trb,
 mr.blk,
 mr.stl,
 mr.tov,
 mr.pf,
 mr.drb,
 mr.orb,
 mr.fg,
 mr.fga,
 mr.fg_pct,
 mr.fg2,
 mr.fg2a,
 mr.fg2_pct,
 mr.fg3,
 mr.fg3a,
 mr.fg3_pct,
 mr.ft,
 mr.fta,
 mr.ft_pct,
 mr.away_g,
 mr.away_mp,
 mr.away_pts,
 mr.away_ast,
 mr.away_trb,
 mr.away_blk,
 mr.away_stl,
 mr.away_tov,
 mr.away_pf,
 mr.away_drb,
 mr.away_orb,
 mr.away_fg,
 mr.away_fga,
 mr.away_fg_pct,
 mr.away_fg2,
 mr.away_fg2a,
 mr.away_fg2_pct,
 mr.away_fg3,
 mr.away_fg3a,
 mr.away_fg3_pct,
 mr.away_ft,
 mr.away_fta,
 mr.away_fta_pct
FROM combined_data_inbound AS cdi 
LEFT OUTER JOIN model_ready AS mr 
ON cdi.game_id = mr.game_id 
WHERE cdi.game_id LIKE '20220214%%'"""

In [4]:
df = pd.read_sql(sql=query, con=connection)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 88 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   game_id                9 non-null      object        
 1   game_date              9 non-null      datetime64[ns]
 2   home_team              9 non-null      object        
 3   away_team              9 non-null      object        
 4   home_score             9 non-null      int64         
 5   away_score             9 non-null      int64         
 6   home_result            0 non-null      object        
 7   covers_game_url        9 non-null      object        
 8   league_year            0 non-null      object        
 9   home_spread            0 non-null      object        
 10  home_spread_result     0 non-null      object        
 11  pred_date              9 non-null      datetime64[ns]
 12  open_line_home         9 non-null      float64       
 13  open_line

In [6]:
df.head(10)

,game_id,game_date,home_team,away_team,home_score,away_score,home_result,covers_game_url,league_year,home_spread,home_spread_result,pred_date,open_line_home,open_line_away,fd_line_price_home,fd_line_away,fd_line_price_away,dk_line_price_home,dk_line_away,dk_line_price_away,covers_consenses_away,home_team_num,away_team_num,league_year_end,home_line,fd_line_home,dk_line_home,covers_consenses_home,wins,losses,win_pct,expected_wins,expected_losses,home_ppg,home_papg,away_wins,away_losses,away_win_pct,away_expected_wins,away_expected_losses,away_ppg,away_papg,g,mp,pts,ast,trb,blk,stl,tov,pf,drb,orb,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,away_g,away_mp,away_pts,away_ast,away_trb,away_blk,away_stl,away_tov,away_pf,away_drb,away_orb,away_fg,away_fga,away_fg_pct,away_fg2,away_fg2a,away_fg2_pct,away_fg3,away_fg3a,away_fg3_pct,away_ft,away_fta,away_fta_pct
0,20220214WASDET,2022-02-14 19:00:00,WAS,DET,103,94,None,/sport/basketball/nba/matchup/251411,None,None,None,2022-02-13,-4.5,4.5,-110,4.0,-110,-105,4.0,-115,0.27,30,9,0,-4.5,-4.0,-4.0,0.73,25.0,30.0,0.455,22.0,33.0,106.9,110.2,12.0,44.0,0.214,12.0,44.0,102.5,112.8,55.0,13300.0,5881.0,1321.0,2412.0,270.0,342.0,697.0,1032.0,1907.0,505.0,2191.0,4730.0,0.463,1640.0,3025.0,0.542,551.0,1705.0,0.323,948.0,1216.0,0.780,56.0,13490.0,6317.0,1433.0,2638.0,288.0,428.0,803.0,1129.0,2061.0,577.0,2290.0,4825.0,0.475,1594.0,2970.0,0.537,696.0,1855.0,0.375,1041.0,1347.0,0.773
1,20220214BKNSAC,2022-02-14 19:30:00,BKN,SAC,109,85,None,/sport/basketball/nba/matchup/251412,None,None,None,2022-02-13,3.5,-3.5,-112,-2.5,-108,-110,-2.0,-110,0.49,2,26,0,3.5,2.5,2.0,0.51,29.0,27.0,0.518,28.0,28.0,110.9,111.2,22.0,36.0,0.379,21.0,37.0,110.2,114.8,56.0,13490.0,6213.0,1399.0,2503.0,302.0,397.0,726.0,1115.0,1934.0,569.0,2320.0,4960.0,0.468,1694.0,3175.0,0.534,626.0,1785.0,0.351,947.0,1169.0,0.810,58.0,13995.0,6658.0,1442.0,2656.0,299.0,462.0,712.0,1109.0,2019.0,637.0,2507.0,5287.0,0.474,1786.0,3291.0,0.543,721.0,1996.0,0.361,923.0,1199.0,0.770
2,20220214NYKOKC,2022-02-14 19:30:00,NYK,OKC,123,127,None,/sport/basketball/nba/matchup/251413,None,None,None,2022-02-13,-8.0,8.0,-110,10.0,-110,-110,10.0,-110,0.54,20,21,0,-8.0,-10.0,-10.0,0.46,25.0,32.0,0.439,26.0,31.0,104.7,106.1,17.0,39.0,0.304,16.0,40.0,100.7,107.5,57.0,13755.0,5969.0,1220.0,2618.0,277.0,394.0,730.0,1133.0,1970.0,648.0,2133.0,4910.0,0.434,1394.0,2817.0,0.495,739.0,2093.0,0.353,964.0,1288.0,0.748,56.0,13490.0,6018.0,1399.0,2730.0,344.0,453.0,718.0,985.0,2111.0,619.0,2244.0,5020.0,0.447,1515.0,2890.0,0.524,729.0,2130.0,0.342,801.0,1067.0,0.751
3,20220214CHISAS,2022-02-14 20:00:00,CHI,SAS,120,109,None,/sport/basketball/nba/matchup/251414,None,None,None,2022-02-13,-4.5,4.5,-115,3.5,-105,-115,3.5,-105,0.28,5,27,0,-4.5,-3.5,-3.5,0.72,36.0,21.0,0.632,32.0,25.0,112.3,110.3,22.0,35.0,0.386,29.0,28.0,111.8,111.6,57.0,13705.0,6400.0,1394.0,2458.0,259.0,407.0,703.0,1063.0,1955.0,503.0,2389.0,4957.0,0.482,1749.0,3250.0,0.538,640.0,1707.0,0.375,982.0,1207.0,0.814,57.0,13755.0,6362.0,1358.0,2669.0,298.0,422.0,750.0,1038.0,2020.0,649.0,2417.0,5205.0,0.464,1751.0,3318.0,0.528,666.0,1887.0,0.353,862.0,1130.0,0.763
4,20220214MILPOR,2022-02-14 20:00:00,MIL,POR,107,122,None,/sport/basketball/nba/matchup/251415,None,None,None,2022-02-13,-15.5,15.5,-112,14.5,-108,-115,15.0,-105,0.53,17,25,0,-15.5,-14.5,-15.0,0.47,35.0,22.0,0.614,35.0,22.0,112.9,109.2,23.0,34.0,0.404,21.0,36.0,107.7,112.0,57.0,13730.0,6435.0,1336.0,2653.0,241.0,441.0,742.0,1027.0,2067.0,586.0,2332.0,5073.0,0.460,1520.0,2834.0,0.536,812.0,2239.0,0.363,959.0,1235.0,0.777,57.0,13705.0,6384.0,1487.0,2496.0,265.0,427.0,744.0,1119.0,1974.0,522.0,2326.0,4912.0,0.474,1543.0,2811.0,0.549,783.0,2101.0,0.373,949.0,1252.0,0.758
5,20220214NOPTOR,2022-02-14 20:00:00,NOP,TOR,120,90,None,/sport/basketball/nba/matchup/251416,None,None,None,2022-02-13,4.0,-4.0,-110,-3.5,-110,-105,-3.0,-115,0.74,19,28,0,4.0,3.5,3.0,0.26,22.0,34.0,0.393,22.0,34.0,105.8,109.4,31.0,24.0,0.564,31.0,24.0,109.1,106.9,56.0,13490.0,5

<a id=model_setup></a>
## Load Models

### AutoML

In [7]:
AutoML_model = load_model('../models/AutoML/Baseline_Ridge_Reg_PyCaret')

Transformation Pipeline and Model Successfully Loaded


### DL

In [8]:
DL_model = keras.models.load_model('../models/Deep_Learning/original_baseline_2022-01-20_01:59:27')

## Model Predictions

TODO: Improve choice of inbound features. (iloc[:, 21:])  
TODO: Functionality for diff models with diff features. (with or without fd and dk lines)

In [9]:
df.iloc[:, 21:]

,home_team_num,away_team_num,league_year_end,home_line,fd_line_home,dk_line_home,covers_consenses_home,wins,losses,win_pct,expected_wins,expected_losses,home_ppg,home_papg,away_wins,away_losses,away_win_pct,away_expected_wins,away_expected_losses,away_ppg,away_papg,g,mp,pts,ast,trb,blk,stl,tov,pf,drb,orb,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,away_g,away_mp,away_pts,away_ast,away_trb,away_blk,away_stl,away_tov,away_pf,away_drb,away_orb,away_fg,away_fga,away_fg_pct,away_fg2,away_fg2a,away_fg2_pct,away_fg3,away_fg3a,away_fg3_pct,away_ft,away_fta,away_fta_pct
0,30,9,0,-4.5,-4.0,-4.0,0.73,25.0,30.0,0.455,22.0,33.0,106.9,110.2,12.0,44.0,0.214,12.0,44.0,102.5,112.8,55.0,13300.0,5881.0,1321.0,2412.0,270.0,342.0,697.0,1032.0,1907.0,505.0,2191.0,4730.0,0.463,1640.0,3025.0,0.542,551.0,1705.0,0.323,948.0,1216.0,0.780,56.0,13490.0,6317.0,1433.0,2638.0,288.0,428.0,803.0,1129.0,2061.0,577.0,2290.0,4825.0,0.475,1594.0,2970.0,0.537,696.0,1855.0,0.375,1041.0,1347.0,0.773
1,2,26,0,3.5,2.5,2.0,0.51,29.0,27.0,0.518,28.0,28.0,110.9,111.2,22.0,36.0,0.379,21.0,37.0,110.2,114.8,56.0,13490.0,6213.0,1399.0,2503.0,302.0,397.0,726.0,1115.0,1934.0,569.0,2320.0,4960.0,0.468,1694.0,3175.0,0.534,626.0,1785.0,0.351,947.0,1169.0,0.810,58.0,13995.0,6658.0,1442.0,2656.0,299.0,462.0,712.0,1109.0,2019.0,637.0,2507.0,5287.0,0.474,1786.0,3291.0,0.543,721.0,1996.0,0.361,923.0,1199.0,0.770
2,20,21,0,-8.0,-10.0,-10.0,0.46,25.0,32.0,0.439,26.0,31.0,104.7,106.1,17.0,39.0,0.304,16.0,40.0,100.7,107.5,57.0,13755.0,5969.0,1220.0,2618.0,277.0,394.0,730.0,1133.0,1970.0,648.0,2133.0,4910.0,0.434,1394.0,2817.0,0.495,739.0,2093.0,0.353,964.0,1288.0,0.748,56.0,13490.0,6018.0,1399.0,2730.0,344.0,453.0,718.0,985.0,2111.0,619.0,2244.0,5020.0,0.447,1515.0,2890.0,0.524,729.0,2130.0,0.342,801.0,1067.0,0.751
3,5,27,0,-4.5,-3.5,-3.5,0.72,36.0,21.0,0.632,32.0,25.0,112.3,110.3,22.0,35.0,0.386,29.0,28.0,111.8,111.6,57.0,13705.0,6400.0,1394.0,2458.0,259.0,407.0,703.0,1063.0,1955.0,503.0,2389.0,4957.0,0.482,1749.0,3250.0,0.538,640.0,1707.0,0.375,982.0,1207.0,0.814,57.0,13755.0,6362.0,1358.0,2669.0,298.0,422.0,750.0,1038.0,2020.0,649.0,2417.0,5205.0,0.464,1751.0,3318.0,0.528,666.0,1887.0,0.353,862.0,1130.0,0.763
4,17,25,0,-15.5,-14.5,-15.0,0.47,35.0,22.0,0.614,35.0,22.0,112.9,109.2,23.0,34.0,0.404,21.0,36.0,107.7,112.0,57.0,13730.0,6435.0,1336.0,2653.0,241.0,441.0,742.0,1027.0,2067.0,586.0,2332.0,5073.0,0.460,1520.0,2834.0,0.536,812.0,2239.0,0.363,959.0,1235.0,0.777,57.0,13705.0,6384.0,1487.0,2496.0,265.0,427.0,744.0,1119.0,1974.0,522.0,2326.0,4912.0,0.474,1543.0,2811.0,0.549,783.0,2101.0,0.373,949.0,1252.0,0.758
5,19,28,0,4.0,3.5,3.0,0.26,22.0,34.0,0.393,22.0,34.0,105.8,109.4,31.0,24.0,0.564,31.0,24.0,109.1,106.9,56.0,13490.0,5927.0,1378.0,2527.0,237.0,424.0,758.0,1102.0,1883.0,644.0,2164.0,4883.0,0.443,1563.0,3043.0,0.514,601.0,1840.0,0.327,998.0,1257.0,0.794,55.0,13325.0,5882.0,1364.0,2436.0,274.0,387.0,854.0,1037.0,1849.0,587.0,2132.0,4640.0,0.459,1452.0,2723.0,0.533,680.0,1917.0,0.355,938.0,1174.0,0.799
6,8,22,0,-11.0,-11.0,-11.0,0.77,31.0,25.0,0.554,32.0,24.0,109.6,107.6,13.0,45.0,0.224,15.0,43.0,103.1,111.5,56.0,13490.0,6135.0,1520.0,2449.0,209.0,402.0,755.0,1079.0,1940.0,509.0,2289.0,4843.0,0.473,1568.0,2801.0,0.560,721.0,2042.0,0.353,836.0,1063.0,0.786,58.0,13945.0,6466.0,1441.0,2719.0,309.0,464.0,734.0,1063.0,2106.0,613.0,2365.0,5148.0,0.459,1622.0,3093.0,0.524,743.0,2055.0,0.362,993.0,1282.0,0.775
7,29,11,0,-14.0,-15.0,-14.5,0.71,35.0,21.0,0.625,39.0,17.0,113.6,107.0,15.0,40.0,0.273,14.0,41.0,108.3,117.3,56.0,13440.0,6361.0,1244.0,2579.0,275.0,398.0,743.0,1054.0,2007.0,572.0,2274.0,4822.0,0.472,1460.0,2560.0,0.570,814.0,2262.0,0.360,999.0,1280.0,0.780,55.0,13225.0,6451.0,1394.0,2467.0,320.0,538.0,721.0,1246.0,1876.0,591.0,2368.0,4932.0,0.480,1644.0,2873.0,0.572,724.0,2059.0,0.352,991.0,1269.0,0.781
8,13,10,0,6.0,5.5,5.5,0.26,28.0,30.0,0.483,26.0,32.0,106.2,107.8,42.0,15.0,0.737,41.0,16.0,110.6,103.2,58.0,13970.0,6161.0,1355.0,2537.0,261.0,434.0,778.0,1091.0,202

### AutoML

In [10]:
automl_predictions = predict_model(AutoML_model, data=df.iloc[:, 21:])

In [11]:
automl_predictions

,home_team_num,away_team_num,league_year_end,home_line,fd_line_home,dk_line_home,covers_consenses_home,wins,losses,win_pct,expected_wins,expected_losses,home_ppg,home_papg,away_wins,away_losses,away_win_pct,away_expected_wins,away_expected_losses,away_ppg,away_papg,g,mp,pts,ast,trb,blk,stl,tov,pf,drb,orb,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,away_g,away_mp,away_pts,away_ast,away_trb,away_blk,away_stl,away_tov,away_pf,away_drb,away_orb,away_fg,away_fga,away_fg_pct,away_fg2,away_fg2a,away_fg2_pct,away_fg3,away_fg3a,away_fg3_pct,away_ft,away_fta,away_fta_pct,Label
0,30,9,0,-4.5,-4.0,-4.0,0.73,25.0,30.0,0.455,22.0,33.0,106.9,110.2,12.0,44.0,0.214,12.0,44.0,102.5,112.8,55.0,13300.0,5881.0,1321.0,2412.0,270.0,342.0,697.0,1032.0,1907.0,505.0,2191.0,4730.0,0.463,1640.0,3025.0,0.542,551.0,1705.0,0.323,948.0,1216.0,0.780,56.0,13490.0,6317.0,1433.0,2638.0,288.0,428.0,803.0,1129.0,2061.0,577.0,2290.0,4825.0,0.475,1594.0,2970.0,0.537,696.0,1855.0,0.375,1041.0,1347.0,0.773,5.64198
1,2,26,0,3.5,2.5,2.0,0.51,29.0,27.0,0.518,28.0,28.0,110.9,111.2,22.0,36.0,0.379,21.0,37.0,110.2,114.8,56.0,13490.0,6213.0,1399.0,2503.0,302.0,397.0,726.0,1115.0,1934.0,569.0,2320.0,4960.0,0.468,1694.0,3175.0,0.534,626.0,1785.0,0.351,947.0,1169.0,0.810,58.0,13995.0,6658.0,1442.0,2656.0,299.0,462.0,712.0,1109.0,2019.0,637.0,2507.0,5287.0,0.474,1786.0,3291.0,0.543,721.0,1996.0,0.361,923.0,1199.0,0.770,5.22971
2,20,21,0,-8.0,-10.0,-10.0,0.46,25.0,32.0,0.439,26.0,31.0,104.7,106.1,17.0,39.0,0.304,16.0,40.0,100.7,107.5,57.0,13755.0,5969.0,1220.0,2618.0,277.0,394.0,730.0,1133.0,1970.0,648.0,2133.0,4910.0,0.434,1394.0,2817.0,0.495,739.0,2093.0,0.353,964.0,1288.0,0.748,56.0,13490.0,6018.0,1399.0,2730.0,344.0,453.0,718.0,985.0,2111.0,619.0,2244.0,5020.0,0.447,1515.0,2890.0,0.524,729.0,2130.0,0.342,801.0,1067.0,0.751,4.86549
3,5,27,0,-4.5,-3.5,-3.5,0.72,36.0,21.0,0.632,32.0,25.0,112.3,110.3,22.0,35.0,0.386,29.0,28.0,111.8,111.6,57.0,13705.0,6400.0,1394.0,2458.0,259.0,407.0,703.0,1063.0,1955.0,503.0,2389.0,4957.0,0.482,1749.0,3250.0,0.538,640.0,1707.0,0.375,982.0,1207.0,0.814,57.0,13755.0,6362.0,1358.0,2669.0,298.0,422.0,750.0,1038.0,2020.0,649.0,2417.0,5205.0,0.464,1751.0,3318.0,0.528,666.0,1887.0,0.353,862.0,1130.0,0.763,4.43590
4,17,25,0,-15.5,-14.5,-15.0,0.47,35.0,22.0,0.614,35.0,22.0,112.9,109.2,23.0,34.0,0.404,21.0,36.0,107.7,112.0,57.0,13730.0,6435.0,1336.0,2653.0,241.0,441.0,742.0,1027.0,2067.0,586.0,2332.0,5073.0,0.460,1520.0,2834.0,0.536,812.0,2239.0,0.363,959.0,1235.0,0.777,57.0,13705.0,6384.0,1487.0,2496.0,265.0,427.0,744.0,1119.0,1974.0,522.0,2326.0,4912.0,0.474,1543.0,2811.0,0.549,783.0,2101.0,0.373,949.0,1252.0,0.758,8.10756
5,19,28,0,4.0,3.5,3.0,0.26,22.0,34.0,0.393,22.0,34.0,105.8,109.4,31.0,24.0,0.564,31.0,24.0,109.1,106.9,56.0,13490.0,5927.0,1378.0,2527.0,237.0,424.0,758.0,1102.0,1883.0,644.0,2164.0,4883.0,0.443,1563.0,3043.0,0.514,601.0,1840.0,0.327,998.0,1257.0,0.794,55.0,13325.0,5882.0,1364.0,2436.0,274.0,387.0,854.0,1037.0,1849.0,587.0,2132.0,4640.0,0.459,1452.0,2723.0,0.533,680.0,1917.0,0.355,938.0,1174.0,0.799,-2.32008
6,8,22,0,-11.0,-11.0,-11.0,0.77,31.0,25.0,0.554,32.0,24.0,109.6,107.6,13.0,45.0,0.224,15.0,43.0,103.1,111.5,56.0,13490.0,6135.0,1520.0,2449.0,209.0,402.0,755.0,1079.0,1940.0,509.0,2289.0,4843.0,0.473,1568.0,2801.0,0.560,721.0,2042.0,0.353,836.0,1063.0,0.786,58.0,13945.0,6466.0,1441.0,2719.0,309.0,464.0,734.0,1063.0,2106.0,613.0,2365.0,5148.0,0.459,1622.0,3093.0,0.524,743.0,2055.0,0.362,993.0,1282.0,0.775,9.04414
7,29,11,0,-14.0,-15.0,-14.5,0.71,35.0,21.0,0.625,39.0,17.0,113.6,107.0,15.0,40.0,0.273,14.0,41.0,108.3,117.3,56.0,13440.0,6361.0,1244.0,2579.0,275.0,398.0,743.0,1054.0,2007.0,572.0,2274.0,4822.0,0.472,1460.0,2560.0,0.570,814.0,2262.0,0.360,999.0,1280.0,0.780,55.0,13225.0,6451.0,1394.0,2467.0,320.0,538.0,721.0,1246.0,1876.0,591.0,2368.0,4932.0,0.480,1644.0,2873.0,0.572,724.0,2059.0,0.352,991.0,1269.0,0.781,12.79197
8,13,10,0,6.0,5.5,5.5,0.26,28.0,30.0,0.483,26.0,32.0,106.2,107.8,42.0,15.0,0.737,41.0,16.0,11

### DL

In [12]:
dl_predictions = DL_model.predict(df.iloc[:, 21:].drop(columns=['fd_line_home', 'dk_line_home', 'covers_consenses_home']))

In [13]:
dl_predictions

array([[-13.727725 ],
       [-22.270153 ],
       [ -4.151995 ],
       [-28.350605 ],
       [  1.1753724],
       [ -5.0788655],
       [ -5.023394 ],
       [ 10.408679 ],
       [ -9.362719 ]], dtype=float32)

In [14]:
dl_predictions = dl_predictions.flatten()

In [15]:
dl_predictions

array([-13.727725 , -22.270153 ,  -4.151995 , -28.350605 ,   1.1753724,
        -5.0788655,  -5.023394 ,  10.408679 ,  -9.362719 ], dtype=float32)

<a id=game_records></a>
## Game Records Table

In [16]:
game_records = pd.DataFrame({'line_hv': 0 - df['home_line'], 'ml_prediction': automl_predictions['Label'], 'dl_prediction': dl_predictions})

### Which Team to Bet On - Home or Away

In [17]:
game_records['is_home_pred_greater_ML'] = game_records['ml_prediction'] > game_records['line_hv']
game_records['is_home_pred_greater_DL'] = game_records['dl_prediction'] > game_records['line_hv']

In [18]:
game_records['ml_pred_direction'] = game_records['is_home_pred_greater_ML'].apply(lambda x: 'Home' if x is True else 'Away')
game_records['dl_pred_direction'] = game_records['is_home_pred_greater_DL'].apply(lambda x: 'Home' if x is True else 'Away')

### Margin Between Line and Prediction

In [19]:
game_records['ml_pred_line_margin'] = game_records.apply(lambda x: x['ml_prediction'] - x['line_hv'] if x['ml_pred_direction'] == 'Home' else x['line_hv'] - x['ml_prediction'], axis=1)
game_records['dl_pred_line_margin'] = game_records.apply(lambda x: x['dl_prediction'] - x['line_hv'] if x['dl_pred_direction'] == 'Home' else x['line_hv'] - x['dl_prediction'], axis=1)

<a id=win_pct></a>
### Win Percentage

TODO: Find proper formula/distribution.
TODO: Determine correst phrasing (win percentage, probability, liklihood, etc.)

In [20]:
# Mean and Standard Deviation
mean_ML = np.mean(game_records['ml_pred_line_margin']) # Currently Not Used
std_dev_ML = np.std(game_records['ml_pred_line_margin'])

mean_DL = np.mean(game_records['dl_pred_line_margin']) # Currently Not Used
std_dev_DL = np.std(game_records['dl_pred_line_margin'])

In [21]:
norm_dist = stats.norm

In [22]:
game_records['ml_win_prob'] = game_records['ml_pred_line_margin'].apply(lambda x: norm_dist.cdf(x, loc=0, scale=std_dev_ML))
game_records['dl_win_prob'] = game_records['dl_pred_line_margin'].apply(lambda x: norm_dist.cdf(x, loc=0, scale=std_dev_DL))

### Line and Line Price
TODO: Bring in actual line prices. Lower Priority

In [23]:
game_records['home_line'] = df['open_line_home']
game_records['home_line_price'] = -110
game_records['away_line'] = df['open_line_away']
game_records['away_line_price'] = -110

<a id=ev></a>
### Expected Value and Expected Value with Vig
TODO: Use formula actual line price from above instead of hard coded "91" for vig. 

In [24]:
game_records['ml_ev'] = game_records['ml_win_prob'].apply(lambda x: 100*x - 100*(1-x))
game_records['dl_ev'] = game_records['dl_win_prob'].apply(lambda x: 100*x - 100*(1-x))

game_records['ml_ev_vig'] = game_records['ml_win_prob'].apply(lambda x: 91*x - 100*(1-x))
game_records['dl_ev_vig'] = game_records['dl_win_prob'].apply(lambda x: 91*x - 100*(1-x))

### Recommended Bet Direction
TODO: Create logic to choose between competing ML and DL recommendations. Lower Priority

In [25]:
game_records['rec_bet_direction'] = game_records['ml_pred_direction']

<a id=game_score></a>
### Game Score
TODO  
Combines ML and DL predictions, win%, and expected value with other outside factors to create a single final metric on a scale of 1 to 100.

In [26]:
game_records['game_score'] = game_records['ml_ev']

<a id=rec_bet></a>
### Recommended Bet Amount
TODO: Combines game scores with bankroll management plan to create bet amount on each game, each day.

In [27]:
game_records['rec_bet_amount'] = 100

### Predicted Win/Loss
Combines Expected Value with Recommended Bet Amount  
TODO: Add logic to incorporate both ML and DL EV and EV with Vig

In [28]:
game_records['predicted_win_loss'] = (game_records['ml_ev'] / 100) * game_records['rec_bet_amount']

### Additional Game Info

In [29]:
game_records['game_id'] = df['game_id']
game_records['game_info'] = df['covers_game_url']
game_records['date'] = df['game_date'].dt.date
game_records['time'] = df['game_date'].dt.time
game_records['home'] = df['home_team']
game_records['away'] = df['away_team']
# game_records['home_line'] = df['open_line_home']
# game_records['home_line_price'] = -110
# game_records['away_line'] = df['open_line_away']
# game_records['away_line_price'] = -110
# game_records['ml_prediction'] = 0
# game_records['ml_pred_direction'] = 0
# game_records['ml_pred_line_margin'] = 0
# game_records['ml_win_prob'] = 0
# game_records['ml_ev'] = 0
# game_records['ml_ev_vig'] = 0
# game_records['dl_prediction'] = 0
# game_records['dl_pred_direction'] = 0
# game_records['dl_pred_line_margin'] = 0
# game_records['dl_win_prob'] = 0
# game_records['dl_ev'] = 0
# game_records['dl_ev_vig'] = 0
# game_records['game_score'] = 0
# game_records['rec_bet_direction'] = 0
# game_records['rec_bet_amount'] = 0
# game_records['predicted_win_loss'] = 0
game_records['bet_amount'] = None # Update on user input
game_records['bet_direction'] = None # Update on user input
game_records['bet_price'] = None # Update on user input
game_records['bet_location'] = None # Update on user input
game_records['game_result'] = 0  # Update after game end
game_records['bet_result'] = "No Bet" # Update after game end
game_records['bet_win_loss'] = None # Update after game end

### Game Record Cleanup

In [30]:
ordered_cols = ['game_id',
 'game_info',
 'date',
 'time',
 'home',
 'away',
 'home_line',
 'home_line_price',
 'away_line',
 'away_line_price',
 'ml_prediction',
 'ml_pred_direction',
 'ml_pred_line_margin',
 'ml_win_prob',
 'ml_ev',
 'ml_ev_vig',
 'dl_prediction',
 'dl_pred_direction',
 'dl_pred_line_margin',
 'dl_win_prob',
 'dl_ev',
 'dl_ev_vig',
 'game_score',
 'rec_bet_direction',
 'rec_bet_amount',
 'predicted_win_loss',
 'game_result',
 'bet_result',
 'bet_amount',
 'bet_direction',
 'bet_price',
 'bet_location',
 'bet_win_loss']

In [31]:
game_records = game_records[ordered_cols]

In [32]:
game_records

,game_id,game_info,date,time,home,away,home_line,home_line_price,away_line,away_line_price,ml_prediction,ml_pred_direction,ml_pred_line_margin,ml_win_prob,ml_ev,ml_ev_vig,dl_prediction,dl_pred_direction,dl_pred_line_margin,dl_win_prob,dl_ev,dl_ev_vig,game_score,rec_bet_direction,rec_bet_amount,predicted_win_loss,game_result,bet_result,bet_amount,bet_direction,bet_price,bet_location,bet_win_loss
0,20220214WASDET,/sport/basketball/nba/matchup/251411,2022-02-14,19:00:00,WAS,DET,-4.5,-110,4.5,-110,5.64198,Home,1.14198,0.65879,31.75812,25.82901,-13.72773,Away,18.22773,0.97425,94.85078,86.08249,31.75812,Home,100,31.75812,0,No Bet,None,None,None,None,None
1,20220214BKNSAC,/sport/basketball/nba/matchup/251412,2022-02-14,19:30:00,BKN,SAC,3.5,-110,-3.5,-110,5.22971,Home,8.72971,0.99912,99.82387,90.83180,-22.27015,Away,18.77015,0.97753,95.50695,86.70914,99.82387,Home,100,99.82387,0,No Bet,None,None,None,None,None
2,20220214NYKOKC,/sport/basketball/nba/matchup/251413,2022-02-14,19:30:00,NYK,OKC,-8.0,-110,8.0,-110,4.86549,Away,3.13451,0.86930,73.85933,66.03566,-4.15200,Away,12.15200,0.90290,80.58004,72.45394,73.85933,Away,100,73.85933,0,No Bet,None,None,None,None,None
3,20220214CHISAS,/sport/basketball/nba/matchup/251414,2022-02-14,20:00:00,CHI,SAS,-4.5,-110,4.5,-110,4.43590,Away,0.06410,0.50916,1.83222,-2.75023,-28.35061,Away,32.85061,0.99978,99.95512,90.95714,1.83222,Away,100,1.83222,0,No Bet,None,None,None,None,None
4,20220214MILPOR,/sport/basketball/nba/matchup/251415,2022-02-14,20:00:00,MIL,POR,-15.5,-110,15.5,-110,8.10756,Away,7.39244,0.99596,99.19188,90.22825,1.17537,Away,14.32463,0.93704,87.40744,78.97410,99.19188,Away,100,99.19188,0,No Bet,None,None,None,None,None
5,20220214NOPTOR,/sport/basketball/nba/matchup/251416,2022-02-14,20:00:00,NOP,TOR,4.0,-110,-4.0,-110,-2.32008,Home,1.67992,0.72638,45.27603,38.73861,-5.07887,Away,1.07887,0.54588,9.17612,4.26320,45.27603,Home,100,45.27603,0,No Bet,None,None,None,None,None
6,20220214DENORL,/sport/basketball/nba/matchup/251417,2022-02-14,21:00:00,DEN,ORL,-11.0,-110,11.0,-110,9.04414,Away,1.95586,0.75828,51.65533,44.83084,-5.02339,Away,16.02339,0.95654,91.30767,82.69882,51.65533,Away,100,51.65533,0,No Bet,None,None,None,None,None
7,20220214UTAHOU,/sport/basketball/nba/matchup/251418,2022-02-14,21:00:00,UTA,HOU,-14.0,-110,14.0,-110,12.79197,Away,1.20803,0.66743,33.48612,27.47925,10.40868,Away,3.59132,0.64939,29.87828,24.03376,33.48612,Away,100,33.48612,0,No Bet,None,None,None,None,None
8,20220214LACGSW,/sport/basketball/nba/matchup/251419,2022-02-14,22:30:00,LAC,GSW,6.0,-110,-6.0,-110,-3.10555,Home,2.89445,0.85015,70.02931,62.37799,-9.36272,Away,3.36272,0.64030,28.05958,22.29690,70.02931,Home,100,70.02931,0,No Bet,None,None,None,None,None


In [33]:
game_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   game_id              9 non-null      object 
 1   game_info            9 non-null      object 
 2   date                 9 non-null      object 
 3   time                 9 non-null      object 
 4   home                 9 non-null      object 
 5   away                 9 non-null      object 
 6   home_line            9 non-null      float64
 7   home_line_price      9 non-null      int64  
 8   away_line            9 non-null      float64
 9   away_line_price      9 non-null      int64  
 10  ml_prediction        9 non-null      float32
 11  ml_pred_direction    9 non-null      object 
 12  ml_pred_line_margin  9 non-null      float64
 13  ml_win_prob          9 non-null      float64
 14  ml_ev                9 non-null      float64
 15  ml_ev_vig            9 non-null      float64

<a id=save></a>
## Save to RDS

In [34]:
# game_records.to_sql(
#             name="game_records",
#             con=connection,
#             index=False,
#             if_exists="append",
#         )